In [1]:
import os
import sys
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
from tqdm import tqdm

PATH_PROJ = '/home/rustem/kaggle/mts'
sys.path.append(PATH_PROJ)

from src.lib.DBManager import DBManager
from src.base.settings import connector

In [2]:
db = DBManager(connector)

In [3]:
user_ids = db.read_data_as_df(
    """
    select
        distinct user_id
    from
        data
    """
)

In [7]:
list_user_ids = user_ids['user_id'].tolist()

In [19]:
db.read_data_as_df(
    f"""
    select
        *
    from
        data
    where
        user_id in ({', '.join([str(x) for x in list_user_ids[:5000]])})
    """
)

,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id
0,Тюменская область,Тюмень,Apple,iPhone 7,apple.com,smartphone,iOS,35752.0,2021-07-04,night,2,100105
1,Тюменская область,Тюмень,Apple,iPhone 7,apple.com,smartphone,iOS,35752.0,2021-07-12,night,1,100105
2,Тюменская область,Тюмень,Apple,iPhone 7,banners.adfox.ru,smartphone,iOS,35752.0,2021-07-12,evening,1,100105
3,Тюменская область,Тюмень,Apple,iPhone 7,apple.com,smartphone,iOS,35752.0,2021-06-16,evening,4,100105
4,Тюменская область,Тюмень,Apple,iPhone 7,instagram.com,smartphone,iOS,35752.0,2021-06-29,evening,1,100105
...,...,...,...,...,...,...,...,...,...,...,...,...
4464796,Чукотский АО,Анадырь,Huawei,Honor 10 Lite,yastatic.net,smartphone,Android,13528.0,2021-08-19,evening,2,303514
4464797,Ненецкий АО,Нарьян-Мар,Apple,iPhone XR,icloud.com,smartphone,iOS,56843.0,2021-07-19,day,1,197358
4464798,Ненецкий АО,Нарьян-Мар,Apple,iPhone XR,yandex.ru,smartphone,iOS,56843.0,2021-07-19,day,1,197358
4464799,Ненецкий АО,Нарьян-Мар,Apple,iPhone XR,m.facebook.com,smartphone,iOS,56843.0,2021-07-19,day,1,197358


In [3]:
db.read_data_as_df(
    """
    select
        *
    from
        data
    where
        rowid in (1, 2, 3)
    """
)

,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id
0,Тюменская область,Тюмень,Apple,iPhone 7,apple.com,smartphone,iOS,35752.0,2021-07-04,night,2,100105
1,Тюменская область,Тюмень,Apple,iPhone 7,apple.com,smartphone,iOS,35752.0,2021-07-12,night,1,100105
2,Тюменская область,Тюмень,Apple,iPhone 7,banners.adfox.ru,smartphone,iOS,35752.0,2021-07-12,evening,1,100105
